# Import Modules

In [1]:
import pandas as pd
import numpy as np
import time

# Import the Datasets

In [2]:
peopleFilepath = './Data/pickledPeople'
voteFilepath = './Data/pickledVotes_unpacked'
routeFilepath = './Data/pickledRoutes_unpacked'

people = pd.read_pickle(peopleFilepath)
votes = pd.read_pickle(voteFilepath)
routes = pd.read_pickle(routeFilepath)

FileNotFoundError: [Errno 2] No such file or directory: './Data/pickledPeople'

# Define function to find people with common routes

In [3]:
# Function to find all common people between N most-popular routes
def findCommonPeople(areaUrl, breadcrumbTier, nRoutes, routes, people):
    areaRoutes = routes.loc[routes[breadcrumbTier]==areaUrl]
    areaRoutes = areaRoutes.sort_values('numQualityVotes', ascending=False)
    areaRoutes = areaRoutes[:nRoutes]
    
    subPeople = set([])
    
    # Filter the people dataset to only include those with at least nRoutes starRatings
    people = people.loc[people['numStarRatings']>=nRoutes*5]
            
    for i in range(len(areaRoutes)):
        tempSet=[]
        print("Parsing peopleDF for route", i+1, "of", nRoutes)
        for j in range(len(people)):
            if areaRoutes.index[i] in people.iloc[j]['starRatings']:
                tempSet.append(people.index[j])
                print(j/len(people), end='\r')        
        if i==0:
            subPeople=set(tempSet)
        else:
            subPeople = subPeople.intersection(set(tempSet))
            
    print(len(subPeople),"have climbed the", nRoutes, "most-rated climbs at", areaUrl) 

    return subPeople